# Trabajo 3
Alumnos:
- Marcos Hernández Rodríguez
- Salva ***

# Setup

In [20]:
%load_ext autoreload
%autoreload 2

import numpy as np
from modules import config as cf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. Sobre la base de datos de peatones para clasificación. Aplicar un clasificador SVM. Se deberá implementar HoG (no usar una función ya implementada). Obtener resultados de bondad del clasificador
2. Sobre la base de datos de peatones para clasificación. Aplicar un clasificador usando una red neuronal de convolución. Se deberán reusar y modificar las arquitecturas VGG16, ResNet18 y Xception para tal fin. Obtener una comparativa de bondad usando predictores como agudeza, F1, recall, precision.
3. Sobre la base de datos de peatones para clasificación. Compara la mejor red neuronal obtenida en el punto 2 con el clasificador SVM.

# Read Data

In [62]:
# Preparar los datos para las imágenes con personas andando
import cv2
import os

pedestrians_path = 'input/classification/pedestrians128x64/'
pedestrians_images, pedestrians_labels = h.load_images_from_folder(pedestrians_path, class_label=1)
print(type(pedestrians_images))
print(len(pedestrians_images))
print(pedestrians_images[0].shape)


# Select 400 images randomly
import random
pedestrians_images = random.sample(pedestrians_images, 400)
print(type(pedestrians_images))
print(len(pedestrians_images))
print(pedestrians_images[0].shape)

<class 'list'>
924
(128, 64, 3)
<class 'list'>
400
(128, 64, 3)


In [63]:
# esto no se si está bien

import random
random.shuffle(pedestrians_images)
x_train = pedestrians_images[int(len(pedestrians_images)/2):]
x_test = pedestrians_images[:int(len(pedestrians_images)/2)]

#### no peatones

In [65]:
# Preparar los datos para las imágenes sin personas andando
no_pedestrians_path = 'input/classification/pedestrians_neg/'
no_pedestrians_images, no_pedestrians_labels = h.load_images_from_folder(pedestrians_path, class_label=0)
print(type(no_pedestrians_images))
print(len(no_pedestrians_images))
print(no_pedestrians_images[0].shape)

# Select images randomly
import random
no_pedestrians_images = random.sample(no_pedestrians_images, 40)
print(type(no_pedestrians_images))
print(len(no_pedestrians_images))
print(no_pedestrians_images[0].shape)

# Resize de las imagenes a 512x512
import cv2
for i, img in enumerate(no_pedestrians_images):
    no_pedestrians_images[i] = cv2.resize(no_pedestrians_images[i], (512, 512))

    # Escoger un trozo de 128x64
    no_pedestrians_images[i] = no_pedestrians_images[i][192:320, 224:288]

print(no_pedestrians_images[0].shape)
print(len(no_pedestrians_images))

<class 'list'>
924
(128, 64, 3)
<class 'list'>
40
(128, 64, 3)
(128, 64, 3)
40


## Caracterizar la imagen usando Histograma de Orientación de Gradiente (HOG)

In [57]:
from modules import helpers as h

for i, ped_img in enumerate(pedestrians_images):
    print(i)

    ### PRIMERA FASE: Obtener el gradiente de cada píxel
    gradient_matrix = h.get_gradient_matrix(ped_img)
    # print(f'gradient_matrix shape: {gradient_matrix.shape}')
    print(f'gradient_matrix length: {len(gradient_matrix)}')

    ### SEGUNDA FASE: Obtener la división en células
    cell_list = h.split_in_cells(gradient_matrix, cell_size=8)

    ### TERCERA FASE: Obtener histograma por celda
    for cell in cell_list:
        h.get_histogram(cell)

    if i == 1:
        break

0
128
<class 'numpy.ndarray'>
(128, 64, 3)
<class 'int'> 128
<class 'int'> 64
gradient_matrix length: 128
--- split_in_cells ---
gradient_matrix shape: (128, 64, 2)
cell_rows: 16
cell_columns: 8
result length: 0
[0. 0.]
[0. 0.]
[0. 0.]


c:\Users\marco\GitHub\vision_computador\modules\helpers.py:56: RuntimeWarning: invalid value encountered in sqrt
  norm = np.sqrt(0.5 * (brackets))
c:\Users\marco\GitHub\vision_computador\modules\helpers.py:54: RuntimeWarning: divide by zero encountered in double_scalars
  argument = 0.5 * np.arctan(2*gXY / (gXX-gYY))


IndexError: index 2 is out of bounds for axis 0 with size 2

In [19]:
a = np.zeros((4, 4))
print(a)
a[1, 1] = [4,4,4,4]
print(a)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


ValueError: setting an array element with a sequence.

## Data split

In [ ]:
X=np.concatenate([HOG_X_pos,HOG_X_neg])#Concatenación de HOG
y=np.concatenate([y_pos,y_neg]); #Concatenación de valor de la clase
ntrain= round(0.8*len(y));#cardinal del conjunto entrenamiento
ntest = len(x)-ntrain; #cardinal del conjunto test

idx = np.random.permutation(np.arange(len(y))) #obtenemos una permutación
X_train= X[idx[0:ntrain],:]; #Obtenemos barajado el conjunto entrenamiento
y_train=y[idx[0:ntrain]] #igual para el valor de clase
X_test= X[idx[ntrain+1:len(y)],:];#Obtenemos barajado el conjunto test
y_test=y[idx[ntrain+1:len(y)]]; #igual para el valor de clase


NameError: name 'HOG_X_pos' is not defined

# Classification with SVM

In [ ]:
from sklearn.svm import SVC
m = SVC()
m.fit(x_train,y_train)

y_pred_train=m.predict(x_train); 

y_pred_test=m.predict(x_test); 

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred_train)
confusion_matrix(y_test, y_pred_test)

# Classification with CNN

2. Sobre la base de datos de peatones para clasificación. Aplicar un clasificador usando una red neuronal de convolución. Se deberán reusar y modificar las arquitecturas VGG16, ResNet18 y Xception para tal fin. Obtener una comparativa de bondad usando predictores como agudeza, F1, recall, precision.

In [ ]:
# VGG16, ResNet18 y Xception.
# Dijo Sara que se puede usar ResNet50 en vez de la 18 si no funciona bien la 18

# usar transfer learning

### VGG16

In [23]:
import tensorflow.keras as keras
import tensorflow as tf

# Utilizamos el modelo VGG16
base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=cf.input_shape,
    include_top=False,
)

base_model.summary()

# Congelamos capas 
base_model.trainable = False


inputs = keras.Input(shape=cf.input_shape)
data_augmentation = keras.Sequential(
    [
        # Reflejo horizontal
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        # Rotaciones 
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)
x = data_augmentation(inputs)

# Normalización
#usando outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# mean = np.append(mean, mean[0])
# var = np.append(var, var[0])
print(norm_layer.get_weights())
print(var)


x = norm_layer(x)
#norm_layer.set_weights([mean, var])


x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 64, 3)]      0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 64, 64)       1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 64, 64)       36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 16, 128)       0     

In [ ]:
import matplotlib.pyplot as plt

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 5
history = model.fit(train_ds, epochs=epochs, validation_data=validation_ds)


plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### ResNet18

### Xception

3. Sobre la base de datos de peatones para clasificación. Compara la mejor red neuronal obtenida en el punto 2 con el clasificador SVM.

# Semantic Segmentation

Sobre la base de datos de peatones para segmentación semántica. Partiendo de la
arquitectura base probar diferentes modificaciones de esta. Realizar un análisis de las
arquitecturas estudiadas:
- Obtener en este análisis el promedio de IoU (intersection over union) a lo largo del conjunto test sobre las diferentes arquitecturas estudiadas. Obtener la agudeza.
- Tiempo de aprendizaje.
- Dimensión de la representación latente.

In [ ]:
# El alumno/a debe tener en cuenta que las imágenes tienen diferentes dimensiones tanto en el conjunto de entrenamiento como en el conjunto test. Por lo tanto un paso previo será normalizar las imágenes a la misma dimensión.

In [ ]:
# Read data training images
data_segmentation_train_imgs_path = 'input/data_segmentation/train/images/'
data_seg_train_imgs = h.load_images_from_folder(data_segmentation_train_imgs_path)
print(type(data_seg_train_imgs))
print(len(data_seg_train_imgs))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_train_imgs[i].shape)




# Read data training masks
data_segmentation_train_masks_path = 'input/data_segmentation/train/mask/'
data_seg_train_masks = h.load_images_from_folder(data_segmentation_train_masks_path)
print(type(data_seg_train_masks))
print(len(data_seg_train_masks))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_train_masks[i].shape)

<class 'list'>
96
(406, 612, 3)
(378, 745, 3)
(418, 670, 3)
(436, 786, 3)
(454, 767, 3)
<class 'list'>
96
(406, 612, 3)
(378, 745, 3)
(418, 670, 3)
(436, 786, 3)
(454, 767, 3)


In [ ]:
# Read data test images
data_segmentation_test_imgs_path = 'input/data_segmentation/test/images/'
data_seg_test_imgs = h.load_images_from_folder(data_segmentation_test_imgs_path)
print(type(data_seg_test_imgs))
print(len(data_seg_test_imgs))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_test_imgs[i].shape)




# Read data test masks
data_segmentation_test_masks_path = 'input/data_segmentation/test/mask/'
data_seg_test_masks = h.load_images_from_folder(data_segmentation_test_masks_path)
print(type(data_seg_test_masks))
print(len(data_seg_test_masks))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_test_masks[i].shape)

<class 'list'>
74
(536, 559, 3)
(414, 455, 3)
(445, 479, 3)
(397, 396, 3)
(344, 335, 3)
<class 'list'>
74
(536, 559, 3)
(414, 455, 3)
(445, 479, 3)
(397, 396, 3)
(344, 335, 3)


In [ ]:
print(len(data_seg_train_imgs))
print(len(data_seg_train_masks))

print(len(data_seg_test_imgs))
print(len(data_seg_test_masks))

96
96
74
74


In [60]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import InputLayer, ReLU, Conv3D, Conv3DTranspose, Softmax, Dense

# https://keras.io/api/layers/

# Creamos el modelo
model = tf.keras.Sequential()

prueba_train_shape = 43 # cambiar!

# Falla algo de las dimensiones
model.add(InputLayer(input_shape=(prueba_train_shape, 214, 214, 3), name='imageinput'))
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding="same", name='conv_1'))
# salida de capa de 214x214x64
model.add(ReLU(name='relu_1'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding=(0,0,0,0), name='maxpool_1'))
# salida de capa de 107x107x64
model.add(tf.keras.layers.Conv3D(64, (3,3,64), strides=(1,1), padding="same", name='conv_2'))
model.add(ReLU(name='relu_2'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding=(0,0,0,0), name='maxpool_2'))
model.add(tf.keras.layers.Conv3D(64, (3,3,64), strides=(1,1), padding="same", name='conv_3'))
model.add(ReLU(name='relu_3'))
model.add(tf.keras.layers.Conv3D(64, (3,3,64), strides=(1,1), padding="same", name='conv_4'))

# Capas decodificadoras
# cropping es lo mismo que padding?
model.add(Conv3DTranspose(64, (4,4,64), strides=(2, 2), padding=(0,0,0,0), name='transposed-conv_1'))
model.add(Conv3DTranspose(64, (4,4,64), strides=(2, 2), padding=(0,0,0,0), name='transposed-conv_2'))

model.add(tf.keras.layers.Conv3D(2, (1,1,64), strides=(1,1), padding=(0,0,0,0), name='conv_5'))
# A continuación aplicamos una capa softmax con función de perdida entropia cruzada para obtener la clases fondo o peatón.
model.add(Softmax(name='softmax'))
# ? es una dense?
model.add(Dense(1, activation='sigmoid', name='classoutput'))



model.summary()


model.compile(optimizer="adam", loss="BinaryCrossentropy")

2.8.0


ValueError: The `pool_size` argument must be a tuple of 3 integers. Received: (2, 2)